In [1]:
import marvin
marvin.config.download = True
marvin.config.release = 'DR17'

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import scipy.stats as sci

from marvin.tools.vacs import VACs
from astropy.io import fits
import numpy as np
from marvin.tools import Maps
from marvin.tools import Cube

from marvin import config #Needed to fix "Cube not subscriptable" when running in Sciserver
config.forceDbOff() #Needed to fix "Cube not subscriptable" when running in Sciserver

print('Marvin and Matplotlib imported. Warnings about missing paths/files are typical, so no need to be concerned')

/home/idies/miniconda3/envs/py38/lib/python3.8/site-packages/marvin/core/exceptions.py:50: UserWarning: cannot initiate Sentry error reporting: [Errno 6] No such device or address.
  warnings.warn('cannot initiate Sentry error reporting: {0}.'.format(str(ee)),
[INFO]: No release version set. Setting default to DR17
[WARNING]: path /home/idies/workspace/sdss_sas/mangawork/manga/spectro/redux/v3_1_1/drpall-v3_1_1.fits cannot be found. Setting drpall to None. (MarvinUserWarning)
[WARNING]: path /home/idies/workspace/sdss_sas/mangawork/manga/spectro/analysis/v3_1_1/3.1.0/dapall-v3_1_1-3.1.0.fits cannot be found. Setting dapall to None. (MarvinUserWarning)
[WARNING]: This declarative base already contains a class with the same class name and module name as marvin.db.models.SampleModelClasses.MangaTarget, and will be replaced in the string-lookup table. (SAWarning)


Marvin and Matplotlib imported. Warnings about missing paths/files are typical, so no need to be concerned


In [9]:
mangaplifu=[] #insert list of galaxies to run code on

In [3]:
#a function to get appropriate values for arm vs interarm spaxels
def arm_selector (mangaplifu,at=0.5,it=0.3,bins=25,drange=[0,3.0],usable=0.5):
    plate = mangaplifu.split('-')[0]
    ifu = mangaplifu.split('-')[1]
    
    pi3D = 'https://data.sdss.org/sas/dr17/manga/spectro/pipe3d/v3_1_1/3.1.1/'+str(plate)+'/manga-'+str(plate)+'-'+str(ifu)+'.Pipe3D.cube.fits.gz'
    #Getting the FITS files for Pipe3D
    hdu1 = fits.open(pi3D)
    mass = hdu1[1].data[19]
    error=hdu1[1].data[20]
    mask=hdu1[8].data
    gaia=hdu1[7].data
    
    #Getting the GZ3D files
    maps = marvin.tools.Maps(mangaplifu,mode='remote')
    data = maps.vacs.gz3d
    
    #removing bad spaxels
    m=mass*mask
    mask_m=np.where(m==0,np.nan,m)
    
    #removing spaxels with foreground stars
    indexes0=np.array([])
    indexes1=np.array([])
    for i in range(len(gaia)):
        for j in range(len(gaia[i])):
            if gaia[i][j]==1:
                indexes0=np.append(indexes0,i)
                indexes1=np.append(indexes1,j)
    if len(indexes0)!=0:
        for i in indexes0:
            for i in indexes1:
                mask_m[int(i)][int(j)]=np.nan
    
    #selecting arm vs interarm spaxels
    galaxy = data.spiral_mask_spaxel
    #setting up a threshold
    max_a=np.max(galaxy)
    inter=max_a*it
    ar=max_a*at
    arm = np.where(galaxy > ar,1,np.nan)
    interarm = np.where(galaxy < inter,1,np.nan)
    ma = mask_m*arm
    mi= mask_m*interarm
    
    radius = maps.spx_ellcoo_r_re
    
    #getting binned averages
    mask=np.where(np.isnan(ma),0,1)
    mask2=np.where(np.isnan(mi),0,1)
    mask3=np.where(np.isnan(mask_m),0,1)
    sigmass_ma,radius_ma,binnumber = sci.binned_statistic(radius.value[mask!=0],ma[mask!=0],bins=bins,range=drange,statistic='std')
    count_ma,radius_ma,binnumber = sci.binned_statistic(radius.value[mask!=0],ma[mask!=0],bins=bins,range=drange,statistic='count')
    mass_ma,radius_ma,binnumber = sci.binned_statistic(radius.value[mask!=0],ma[mask!=0],bins=bins,range=drange,statistic='median')
    mass_mi,radius_mi,binnnumber = sci.binned_statistic(radius.value[mask2!=0],mi[mask2!=0],bins=bins,range=drange,statistic='median')
    sigmass_mi,radius_mi,binnumber = sci.binned_statistic(radius.value[mask2!=0],mi[mask2!=0],bins=bins,range=drange,statistic='std')
    count_mi,radius_mi,binnumber = sci.binned_statistic(radius.value[mask2!=0],mi[mask2!=0],bins=bins,range=drange,statistic='count')
    sigmass_all,radius_all,binnumber = sci.binned_statistic(radius.value[mask3!=0],mask_m[mask3!=0],bins=bins,range=drange,statistic='std')
    count_all,radius_all,binnumber = sci.binned_statistic(radius.value[mask3!=0],mask_m[mask3!=0],bins=bins,range=drange,statistic='count')
    mass_all,radius_all,binnumber = sci.binned_statistic(radius.value[mask3!=0],mask_m[mask3!=0],bins=bins,range=drange,statistic='median')

    hdu1.close()
    
    return mass_ma,radius_ma,mass_mi,radius_mi,sigmass_ma,sigmass_mi,count_ma,count_mi,sigmass_all,count_all,mass_all

In [10]:
file = open('arm_vs_interarm.txt','a')
file0=np.loadtxt('arm_vs_interarm.txt',dtype=str,unpack=True)
error=[]

for i in mangaplifu[0:]:
    try:
        mass_ma,radius_ma,mass_mi,radius_mi,sigmass_ma,sigmass_mi,count_ma,count_mi,sigmass_all,count_all,mass_all = arm_selector(i,at=0.5,it=0.3)
        f=(10**(mass_ma)-10**(mass_all))/(10**(mass_all))
        dip=(10**(mass_mi))/(10**(mass_all))
        for j in range(len(mass_ma)):
            row=[str(i),' ', str(radius_ma[j]),' ', str(mass_ma[j]),' ', str(mass_mi[j]),' ', str(f[j]),' ',str(sigmass_ma[j]),' ',str(sigmass_mi[j]),' ',str(count_ma[j]),' ',str(count_mi[j]),' ',str(mass_all[j]),' ', str(sigmass_all[j]),' ', str(count_all[j]),' ',str(dip[j])]
            file.writelines(row)
            file.write('\n')
        print('Successful:',i)
    except:
        error.append(i)
        print('Problems with:',i)
    continue
file.close()
print(error)
#Getting the average mass and radius values and the excess in mass values

Successful: 9510-12701
Successful: 9507-12705
Successful: 10511-12705
Successful: 8945-12701
Successful: 8990-6103
Successful: 8262-9102
Successful: 8323-12703
Successful: 11830-12703
Successful: 8445-12701
Successful: 12621-6101
Successful: 8443-9101
Successful: 11018-12702
Successful: 9879-9101
Successful: 8321-12705
Successful: 11954-12704
Successful: 8322-12701
Successful: 8322-12704
Successful: 8442-12703
Successful: 12514-9102
Successful: 8442-9102
Successful: 11758-9101
[]
